In [7]:
from sympy.combinatorics import *
from itertools import combinations
from sympy.interactive import init_printing

from tqdm.notebook import tqdm

from math import factorial
import numpy as np

init_printing(perm_cyclic=True, pretty_print=False) #prints permutations in compact notation
perm_type = permutations.Permutation

# Reproduce (correct) coefficients from project catalog

In [8]:
def gen_ctype(l: dict,n: int):
    '''
    Input:
        l - cycle decompostion as a dictionary including 1 cycles. e.g. {1:2, 2:1} is a transposition in S_4
        n - specifies order of symmetric group
    Output:
        Generates all permutations of specified cycletype as a list
    '''
    
    has_to_be_partition = 0
    for key, value in l.items():
        has_to_be_partition = has_to_be_partition + key*value
    if has_to_be_partition < n:
        print("Invalid cycle type format.")
    else:
        list_with_cycle_type = []
        for i in list(symmetric(n)):
            if i.cycle_structure == l:
                list_with_cycle_type.append(i)            
        return list_with_cycle_type
    
Permutation(3)(0,1).cycle_structure, gen_ctype({1:2,2:1},4)

({1: 2, 2: 1}, [(2 3), (3)(1 2), (1 3), (3)(0 1), (3)(0 2), (0 3)])

In [9]:
def norm(p, cycle_dict = None):
    if cycle_dict == None:
        cycle_dict = p.cycle_structure
    sum_of_lengths = 0
    num_cycles = 0
    for key, value in cycle_dict.items():
        sum_of_lengths += key*value
        num_cycles += value
    return sum_of_lengths-num_cycles

In [10]:
def norm_check(p,q, norm_sum = None):
    if norm_sum == None:
        norm_sum = norm(p) + norm(q)
    return norm(p*q)==norm_sum

In [11]:
def norm_check2(p,q, norm_sum = None):
    if norm_sum == None:
        norm_sum = norm(p) + norm(q)
    p_trans = set(p.transpositions())
    q_trans = set(q.transpositions())
    
    if p_trans.intersection(q_trans) == set([]):
        return norm(p*q)==norm_sum
    return False

In [12]:
def myprod(p_list, q_list, norm_sum = None):
    if norm_sum:
        return [s*t for s in p_list for t in q_list if norm_check2(s,t,norm_sum)]
    return [s*t for s in p_list for t in q_list if norm_check2(s,t)]

In [13]:
# Create element g2 as a list of permutations (sympy permutations)
g2 = gen_ctype({1:2, 2:1},4)
g2

[(2 3), (3)(1 2), (1 3), (3)(0 1), (3)(0 2), (0 3)]

In [14]:
# Generate powers of g2
g2_square = myprod(g2,g2,2)
g2_cube = myprod(g2,g2_square,3)
g2_quad = myprod(g2,g2_cube,4)

In [15]:
for p in g2_square[0:3]:
    print(p.transpositions())

[(1, 3), (1, 2)]
[(1, 2), (1, 3)]
[(0, 1), (2, 3)]


In [16]:
g2_square[0:3]

[(1 2 3), (1 3 2), (0 1)(2 3)]

In [17]:
def memoize(f):
    memo = {}
    def helper(x):
        if x not in memo:            
            memo[x] = f(x)
        return memo[x]
    return helper

In [18]:
def ctype(p: perm_type):
    '''
    Input:
        Sympy permutation
    Output:
        Gives cycletype of permutation as a tuple with i'th coordinate being multiplicity of i-cycles.
    '''
    d = p.cycle_structure
    max_val = max(d.keys())
    L = []
    for i in range(1,max_val+1):
        if i in list(d.keys()):
            L.append(d[i])
        else:
            L.append(0)
    return tuple(L)

ctype(Permutation(3)(1,3)) # has 2 1-cycles (0 and 2) and one 2-cycle

(2, 1)

In [19]:
def decompose(plist: list, progressbar=True):
    '''
    Input: List of permutations
    Output: Decomposes list by occurences of cycletype
    '''
    cycle_count = {}
    for p in tqdm(plist, disable = progressbar):
        p_type = ctype(p)
        if p_type in list(cycle_count.keys()):
            cycle_count[p_type] += 1
        else:
            cycle_count[p_type] = 1
    return cycle_count

print("g2 squared decomp", decompose(g2_square),
      "\n" "g2 cubed decomp", decompose(g2_cube))

g2 squared decomp {(1, 0, 1): 24, (0, 2): 6} 
g2 cubed decomp {(0, 0, 0, 1): 96}


In [20]:
def num_ctype(ctuple):
    '''
    Input: cycle type as a tuple
    Output: number of permutations with given cycle type using 
    '''
    n = sum((i+1)*ctuple[i] for i in range(len(ctuple)))
    numerator = factorial(n)
    denominator = 1
    for i in range(len(ctuple)):
        denominator *= ((i+1)**ctuple[i]) * factorial(ctuple[i])
    return numerator // denominator

num_ctype((2,1))

6

In [37]:
def to_string(d):
    if type(d) == dict:
        s = ""
        for key, value in d.items():
            s += str(value) + to_string(key)
        s += "  +  "
        return s
    s = "g_"
    for i,k in enumerate(d):
        if i!=0 and k!= 0:
            s+= str(i+1) + "^" + str(k) + ","
    return s[0:-1]

In [39]:
def decompose2(plist, string_output = False):
    '''
    Input: list of permutations
    Output: decomposition according to A(d)
    '''
    d = decompose(plist)
    for i in d.keys():
        d[i] //= num_ctype(i)
        
    if string_output:
        return to_string(d)
    return d

In [22]:
print("g2 square", decompose2(g2_square), 
      "\n" "g2 cube", decompose2(g2_cube), 
      "\n" "g2 quad", decompose2(g2_quad))

g2 square {(1, 0, 1): 3, (0, 2): 2} 
g2 cube {(0, 0, 0, 1): 16} 
g2 quad {}


## Output coefficient matrix

In [ ]:
d = 6
g2 = gen_ctype({1:(d-2), 2:1}, d)

g2_powers = [myprod(g2,g2,2)]

for i in tqdm(range(3,3+3)):
    g2_powers.append(myprod(g2,g2_powers[-1],i))

In [ ]:
g2_powers_decomp = []
for i in tqdm(g2_powers):
    g2_powers_decomp.append(decompose2(i))
    print(g2_powers_decomp[-1])

In [ ]:
all_keys = set([])
for decomp in g2_powers_decomp:
    all_keys = all_keys.union(set(decomp.keys()))
all_keys = all_keys - {(d-2, 1), (d,)}
all_keys = list(all_keys)
all_keys.sort()
all_keys

In [ ]:
(n,m) = (len(g2_powers_decomp),len(all_keys))
A = np.zeros((n,m)).astype(int)

for i in range(n):
    for j in range(m):
        try:
            A[i,j] = (g2_powers_decomp[i])[all_keys[j]]
        except:
            pass

A

In [ ]:
np.linalg.matrix_rank(A)

# Relations

In [ ]:
d = 7
generators = []

for i in tqdm(range(2, d//2+2)):
    generators.append(
        gen_ctype({1:(d-i), i:1}, d)
    )

In [ ]:
for i,g1 in enumerate(generators):
    for j,g2 in enumerate(generators[i:]):
        print(i+2,j+2,decompose2(myprod(g1,g2,i+j+2)))

In [ ]:
decompose2(myprod(generators[0], gen_ctype({1:d-4, 2:2}, d)))

In [24]:
def gen_all(d):
    answer = {}
    
    for i in tqdm(symmetric(d), total = factorial(d)):
        C = ctype(i)
        if C in answer.keys():
            (answer[C]).append(i)
        else:
            answer[C] = [i]
    answer.pop((d,),None)
    return answer

def norm2(p_tuple):
    ans = 0
    for i,p in enumerate(p_tuple):
        ans += i*p
    return ans

In [29]:
d=7
basis = gen_all(d)

In [41]:
for k1,v1 in basis.items():
    for k2,v2 in basis.items():
        if k1 >= k2:
            #print(k1,k2, "\n", decompose2(myprod(v1,v2, norm2(k1) + norm2(k2)), string_output = True))
            pass

In [42]:
products = []
for i in range(1,d//2+1):
    p = basis[(d-2*i, i)]
    for j in range(i,d//2+1):
        q = basis[(d-2*j, j)]
        dic = decompose2(myprod(p,q, i + j))
        products.append([i,j, dic])
        print(i,j, to_string(dic))

1 1 3g_3^12g_2^2  +  
1 2 2g_4^13g_2^1,3^13g_2^3  +  
1 3 2g_2^1,4^13g_2^2,3^1  +  
2 2 5g_5^14g_2^1,4^19g_3^26g_2^2,3^1  +  
2 3 3g_6^15g_2^1,5^16g_3^1,4^1  +  
3 3 7g_7^1  +  


In [ ]:
import pickle

#with open('prod_obs.pkl', 'wb') as f:
#    pickle.dump(products, f)

In [ ]:
with open('prod_obs.pkl', 'rb') as f:
    products = pickle.load(f)
for line in products:
    print(line)

In [ ]:
for line in products:
    print(line[0],line[1], list(line[2].values()))

In [ ]:
g23 = basis[(4,3)]
g22 = basis[(6,2)]

In [ ]:
decompose2(myprod(g22,g23,5))

# Trashcan

In [ ]:
def test_ctype(p: perm_type, D):
    '''
    Input:
        p
    '''
    d = p.cycle_structure
    d.pop(1, None)
    if d.keys() == D.keys():
        for key in D.keys():
            if d[key] != D[key]:
                return False
        return True
    return False